In [1]:
import numpy as np

from astropy.table import Column, MaskedColumn, Table, join, vstack

from herschelhelp.filters import get_filter_meta_table

filter_mean_lambda = {
    item['filter_id']: item['mean_wavelength'] for item in
    get_filter_meta_table()
}

# Reading the catalogues

In [2]:
# Master list

ml = Table.read("../../dmu1/dmu1_ml_ELAIS-N1/data/master_catalogue_elais-n1_20170815.fits")
ml.meta = None

In [3]:
# XID+ MIPS24
# There are two catalogues on SWIRE and SERVS coverage that must be 
# merged (they don't overlap).

mips24_servs = Table.read("../../dmu26/dmu26_XID+MIPS_ELAIS-N1/data/dmu26_XID+MIPS_ELAIS-N1_SERVS_cat_20170725.fits")
mips24_servs.meta = None
mips24_swire = Table.read("../../dmu26/dmu26_XID+MIPS_ELAIS-N1/data/dmu26_XID+MIPS_ELAIS-N1_SWIRE_cat_20170725.fits")
mips24_swire.meta = None
mips24_swire.remove_column("flag_mips24")  # duplicated column

xid_mips24 = vstack([mips24_servs, mips24_swire])

# Adding the error column
xid_mips24.add_column(Column(
    data=np.max([xid_mips24['FErr_MIPS_24_u'] - xid_mips24['F_MIPS_24'],
                 xid_mips24['F_MIPS_24'] - xid_mips24['FErr_MIPS_24_l']],
                axis=0),
    name="ferr_mips_24"
))
xid_mips24['F_MIPS_24'].name = "f_mips_24"
xid_mips24 = xid_mips24['help_id', 'f_mips_24', 'ferr_mips_24', 'flag_mips_24']

In [4]:
# XID+ PACS
# There are two catalogues on SWIRE and SERVS coverage that must be 
# merged (they don't overlap).

pacs_swire = Table.read("../../dmu26/dmu26_XID+PACS_ELAIS-N1/data/dmu26_XID+PACS_ELAIS-N1_SWIRE_cat_20170808.fits")
pacs_swire.meta = None
pacs_servs = Table.read("../../dmu26/dmu26_XID+PACS_ELAIS-N1/data/dmu26_XID+PACS_ELAIS-N1_SERVS_cat_20170808.fits")
pacs_servs.meta = None

xid_pacs = vstack([pacs_servs, pacs_swire])

# Convert from mJy to μJy
for col in ["F_PACS_100", "FErr_PACS_100_u", "FErr_PACS_100_l",
            "F_PACS_160", "FErr_PACS_160_u", "FErr_PACS_160_l"]:
    xid_pacs[col] *= 1000

xid_pacs.add_column(Column(
    data=np.max([xid_pacs['FErr_PACS_100_u'] - xid_pacs['F_PACS_100'],
                 xid_pacs['F_PACS_100'] - xid_pacs['FErr_PACS_100_l']],
                axis=0),
    name="ferr_pacs_green"
))
xid_pacs['F_PACS_100'].name = "f_pacs_green"
xid_pacs['flag_PACS_100'].name = "flag_pacs_green"

xid_pacs.add_column(Column(
    data=np.max([xid_pacs['FErr_PACS_160_u'] - xid_pacs['F_PACS_160'],
                 xid_pacs['F_PACS_160'] - xid_pacs['FErr_PACS_160_l']],
                axis=0),
    name="ferr_pacs_red"
))
xid_pacs['F_PACS_160'].name = "f_pacs_red"
xid_pacs['flag_PACS_160'].name = "flag_pacs_red"

xid_pacs = xid_pacs['help_id', 'f_pacs_green', 'ferr_pacs_green',
                    'flag_pacs_green', 'f_pacs_red', 'ferr_pacs_red',
                    'flag_pacs_red']


In [5]:
# XID+ SPIRE
# There are two catalogues on SWIRE and SERVS coverage that must be 
# merged (they don't overlap).

spire_servs = Table.read("../../dmu26/dmu26_XID+SPIRE_ELAIS-N1/data/dmu26_XID+SPIRE_ELAIS-N1_SERVS_cat_20170725.fits")
spire_servs.meta = None
spire_swire = Table.read("../../dmu26/dmu26_XID+SPIRE_ELAIS-N1/data/dmu26_XID+SPIRE_ELAIS-N1_SWIRE_cat_20170808.fits")
spire_swire.meta = None

xid_spire = vstack([spire_servs, spire_swire])

# Convert from mJy to μJy
for col in ["F_SPIRE_250", "FErr_SPIRE_250_u", "FErr_SPIRE_250_l",
            "F_SPIRE_350", "FErr_SPIRE_350_u", "FErr_SPIRE_350_l",
            "F_SPIRE_500", "FErr_SPIRE_500_u", "FErr_SPIRE_500_l"]:
    xid_spire[col] *= 1000

xid_spire.add_column(Column(
    data=np.max([xid_spire['FErr_SPIRE_250_u'] - xid_spire['F_SPIRE_250'],
                 xid_spire['F_SPIRE_250'] - xid_spire['FErr_SPIRE_250_l']],
                axis=0),
    name="ferr_spire_250"
))
xid_spire['F_SPIRE_250'].name = "f_spire_250"
xid_spire.add_column(Column(
    data=np.max([xid_spire['FErr_SPIRE_350_u'] - xid_spire['F_SPIRE_350'],
                 xid_spire['F_SPIRE_350'] - xid_spire['FErr_SPIRE_350_l']],
                axis=0),
    name="ferr_spire_350"
))
xid_spire['F_SPIRE_350'].name = "f_spire_350"
xid_spire.add_column(Column(
    data=np.max([xid_spire['FErr_SPIRE_500_u'] - xid_spire['F_SPIRE_500'],
                 xid_spire['F_SPIRE_500'] - xid_spire['FErr_SPIRE_500_l']],
                axis=0),
    name="ferr_spire_500"
))
xid_spire['F_SPIRE_500'].name = "f_spire_500"

xid_spire = xid_spire['help_id',
                      'f_spire_250', 'ferr_spire_250', 'flag_spire_250',
                      'f_spire_350', 'ferr_spire_350', 'flag_spire_350',
                      'f_spire_500', 'ferr_spire_500', 'flag_spire_500']

In [6]:
# Photo-z

photoz = Table.read(
    "../../dmu24/dmu24_ELAIS-N1/data/master_catalogue_elais-n1_20170706_photoz_20170725_irac1_optimised.fits")
photoz.meta = None

photoz = photoz['help_id', 'z1_median']
photoz['z1_median'].name = 'redshift'

photoz['redshift'][photoz['redshift'] < 0] = np.nan  # Eazy uses -99 for missing values

In [7]:
# Cigale outputs
cigale = Table.read("../../dmu28/dmu28_ELAIS-N1/data/HELP_final_results.fits")
cigale['id'].name = "help_id"

# We convert the various Chi2 and threshold to flags
flag_cigale_opt = cigale["UVoptIR_OPTchi2"] <= cigale["UVoptIR_OPTchi2_threshold"]
flag_cigale_ir = cigale["UVoptIR_IRchi2"] <= cigale["UVoptIR_IRchi2_threshold"]
flag_cigale = (
    (cigale["UVoptIR_best.reduced_chi_square"] <=  cigale["UVoptIR_best.reduced_chi_square_threshold"]) &
    flag_cigale_opt & flag_cigale_ir)
flag_cigale_ironly = cigale["IRonly_IRchi2"] <= cigale["IRonly_IRchi2_threshold"]

cigale.add_columns([
    MaskedColumn(flag_cigale, "flag_cigale", dtype=int, fill_value=-1),
    MaskedColumn(flag_cigale_opt, "flag_cigale_opt", dtype=int, fill_value=-1),
    MaskedColumn(flag_cigale_ir, "flag_cigale_ir", dtype=int, fill_value=-1),
    MaskedColumn(flag_cigale_ironly, "flag_cigale_ironly", dtype=int, fill_value=-1)
])

cigale['UVoptIR_bayes.stellar.m_star'].name =  "cigale_mstar"
cigale['UVoptIR_bayes.stellar.m_star_err'].name = "cigale_mstar_err"
cigale['UVoptIR_bayes.sfh.sfr10Myrs'].name = "cigale_sfr"
cigale['UVoptIR_bayes.sfh.sfr10Myrs_err'].name = "cigale_sfr_err"
cigale['UVoptIR_bayes.dust.luminosity'].name = "cigale_dustlumin"
cigale['UVoptIR_bayes.dust.luminosity_err'].name = "cigale_dustlumin_err"
cigale['IR_bayes.dust.luminosity'].name = "cigale_dustlumin_ironly"
cigale['IR_bayes.dust.luminosity_err'].name = "cigale_dustlumin_ironly_err"

cigale = cigale['help_id', 'cigale_mstar', 'cigale_mstar_err', 'cigale_sfr', 'cigale_sfr_err',
               'cigale_dustlumin', 'cigale_dustlumin_err', 'cigale_dustlumin_ironly', 'cigale_dustlumin_ironly_err',
               'flag_cigale', 'flag_cigale_opt', 'flag_cigale_ir', 'flag_cigale_ironly']

# Merging

In [8]:
merged_table = join(ml, xid_mips24, join_type='left')
for col in merged_table.colnames:
    if 'flag' in col:
        merged_table[col].fill_value = False

merged_table = join(merged_table, xid_pacs, join_type='left')
for col in merged_table.colnames:
    if 'flag' in col:
        merged_table[col].fill_value = False

merged_table = join(merged_table, xid_spire, join_type='left')
for col in merged_table.colnames:
    if 'flag' in col:
        merged_table[col].fill_value = False

merged_table = join(merged_table, photoz, join_type='left')

merged_table = join(merged_table, cigale, join_type='left')

# Sorting columns

In [9]:
bands = [col[2:] for col in merged_table.colnames
         if col.startswith('f_') and not col.startswith('f_ap')]
bands.sort(key=lambda x: filter_mean_lambda[x])

In [10]:
columns = ['help_id', 'field', 'ra', 'dec', 'hp_idx', 'ebv', 'redshift', 'zspec']
for band in bands:
    for col_tpl in ['f_{}', 'ferr_{}', 'f_ap_{}', 'ferr_ap_{}',
                    'm_{}', 'merr_{}', 'm_ap_{}', 'merr_ap_{}',
                    'flag_{}']:
        colname = col_tpl.format(band)
        if colname in merged_table.colnames:
            columns.append(colname)
columns += ['cigale_mstar', 'cigale_mstar_err', 'cigale_sfr', 'cigale_sfr_err',
            'cigale_dustlumin', 'cigale_dustlumin_err', 'cigale_dustlumin_ironly', 'cigale_dustlumin_ironly_err',
            'flag_cigale', 'flag_cigale_opt', 'flag_cigale_ir', 'flag_cigale_ironly',
            'stellarity', 'flag_cleaned', 'flag_merged', 'flag_gaia', 
            'flag_optnir_obs', 'flag_optnir_det', 'zspec_qual', 'zspec_association_flag']

In [11]:
# Check that we did not forget any column
assert set(columns) == set(merged_table.colnames)


# Saving

In [12]:
merged_table[columns].write("data/ELAIS-N1_20171006.fits")